Snippets:
`(pvio)> python3 -m pip install -r reqs.txt` - zaktualizuj biblioteki pyenv'a

LOAD `((N1 x *.jpg; N2 x *.csv[label_x_ref,*labels_cls]) -> (dy: pandas.dataframe[label_x_ref,*labels_cls], dx: {dy[label_x_ref]: ndarray[N3, *data_x_target_res, 3]]}))`

In [44]:
import glob
import pandas as pd
import cv2
import numpy as np

data_x_path_regex = r"../ISIC2018_Task3_Validation_Input/*.jpg"
data_y_path_regex = r"../ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv"
data_x_target_res = (450, 600)
label_x_ref = 'image'
labels_cls = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']

# preprocess X data
def data_x_loader(data_x_path):
    x = data_x_path
    x = cv2.imread(x)
    x = cv2.resize(x, data_x_target_res)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2YCrCb)
    return x

def load(data_x_path_regex, data_y_path_regex):
    # read & merge Y data
    data_y_paths = glob.glob(data_y_path_regex)
    assert data_y_paths.__len__() > 0
    dy_full = pd.concat([pd.read_csv(data_y_path) for data_y_path in data_y_paths])

    # read & filter_out by Y & map X data
    data_x_paths = glob.glob(data_x_path_regex)
    map_ref2path_full = {ref: next((dxp for dxp in data_x_paths if ref in dxp), None) for ref in dy_full['image']}
    map_ref2path = {ref: data_x_path for ref, data_x_path in map_ref2path_full.items() if data_x_path is not None}
    dx = {ref: data_x_loader(data_x_path) for ref, data_x_path in map_ref2path.items()}

    # filter out Y entries that dont have X entry
    dy = dy_full.loc[dy_full['image'].isin(map_ref2path.keys())]

    print(
    f"""Read stats:
    # entries ok: {dy.__len__()}
    # excess X: {map_ref2path_full.__len__() - map_ref2path.__len__()}
    # excess Y: {dy_full.__len__() - dy.__len__()}
    """)
    return dx, dy

dx, dy = load(data_x_path_regex, data_y_path_regex)
dy

Read stats:
    # entries ok: 193
    # excess X: 0
    # excess Y: 0
    


,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0034321,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0034322,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0034323,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,ISIC_0034324,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0034325,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
188,ISIC_0034519,0.0,1.0,0.0,0.0,0.0,0.0,0.0
189,ISIC_0034520,0.0,1.0,0.0,0.0,0.0,0.0,0.0
190,ISIC_0034521,0.0,0.0,0.0,0.0,1.0,0.0,0.0
191,ISIC_0034522,0.0,1.0,0.0,0.0,0.0,0.0,0.0


TRAIN TEST SPLIT

In [41]:
from sklearn.model_selection import train_test_split
# from sklearn.decomposition import sparse_encode
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

ohe = OneHotEncoder(sparse=False)
# ohe.fit([[label_cls, lix] for lix, label_cls in enumerate(labels_cls)])
ohe.fit(np.reshape(labels_cls, (-1, 1)))
# ohe.fit_transform(labels_cls)
# print(ohe.get_feature_names_out())


tts_factor = .2

def data2clf_data(dy_selected, dx):
    pass

sparse = ohe.inverse_transform(dy[labels_cls])
dyt, dys = train_test_split(dy, test_size=tts_factor, stratify=sparse)



CNN (control case) `(dx, dy) -> clf`

In [43]:
# ohe.inverse_transform(dy[labels_cls].to_numpy())
ohe.transform(ohe.inverse_transform(dy[labels_cls]))

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [2]:
# [y for y in dy['image']]
# data_x_paths
# dy.at[8, 'DF'] = dx[1]
# np.where(dy['image'][0] in data_x_paths)
# for a, in map(str.__contains__, dy['image'], data_x_paths):
#     print(a) 
# dy.loc[dy['image'].isin(['ISIC_0034321', ''])]
# map_ref2path.keys()
# dx
# import keras
# sparse_encode([[0, 1], [1, 0]])


TypeError: sparse_encode() missing 1 required positional argument: 'dictionary'